# PFAS Data Request

### Request for Jireh Data from 9/11/2023 up to date and all data for JAE

### Libraries Import

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

### Data Import and Export

#### Define Functions

In [2]:
def read_excel_sheet(excel_file, sheet_name):
    """
    Read a specific sheet from an Excel file into a DataFrame.

    Args:
        excel_file (str): The path to the Excel file.
        sheet_name (str): The name of the sheet to read.

    Returns:
        pd.DataFrame or None: A DataFrame containing the sheet data or None if the sheet is not found.
    """
    try:
        # Load the specified sheet from the Excel file into a DataFrame
        df = pd.read_excel(excel_file, sheet_name=sheet_name, skiprows=1)
        return df
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

In [3]:
def dfs_to_excel(dfs, filename):
    """
    Write an array of dataframes to separate tabs in an Excel file.
    
    Parameters:
    - dfs: A dictionary where keys are the tab names and values are the DataFrames to write.
    - filename: The name of the Excel file to write to.
    """
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        for tab_name, df in dfs.items():
            # Write each dataframe to a separate sheet
            df.to_excel(writer, sheet_name=tab_name)
    
    print(f"File '{filename}' has been written with {len(dfs)} tabs.")

#### Import Data from Excel

In [5]:
# Example usage:

#Specify where the file is saved use '/' instead of '\'
excel_file_path = 'U:/Research_and_Innovation/_ModellingGroupProjects/PFAS Investigations/Data and analysis/Database as of 06-25-2024.xlsm'
#Specify sheet name
sheet_name = 'PFAS Database'

#Call data import function
data_df = read_excel_sheet(excel_file_path, sheet_name)

data_df.head()

,Sampling point Date Component,Sample Number,Sampling Point,Sample Name,Sampled Date,Analysis,Component,Qualifiers,Result,Units,...,ChainLength,Pseudonym,Dominant Contributor,Sector,Depth,Edit Notes,Field,Notes,Region,Crop
0,ANODIZE SOLUTIONS (FPT)-43692.3333333333-10:2F...,259452.0,ANODIZE SOLUTIONS (FPT),NaN,2019-08-15 08:00:00,PFAS-CL,10:2Fluorotelomersulfonic Acid (Report),NaN,<9.40,ppt_wt_v,...,12.0,Metal Finisher B,NaN,Metal Finishing,NaN,NaN,NaN,NaN,NaN,NaN
1,ANODIZE SOLUTIONS (FPT)-43692.3333333333-11Cl-...,259452.0,ANODIZE SOLUTIONS (FPT),NaN,2019-08-15 08:00:00,PFAS-CL,11Cl-PF3OUdS (Report),NaN,<4.70,ppt_wt_v,...,10.0,Metal Finisher B,NaN,Metal Finishing,NaN,NaN,NaN,NaN,NaN,NaN
2,ANODIZE SOLUTIONS (FPT)-43692.3333333333-4:2Fl...,259452.0,ANODIZE SOLUTIONS (FPT),NaN,2019-08-15 08:00:00,PFAS-CL,4:2Fluorotelomersulfonic Acid (Report),NaN,<4.70,ppt_wt_v,...,6.0,Metal Finisher B,NaN,Metal Finishing,NaN,NaN,NaN,NaN,NaN,NaN
3,ANODIZE SOLUTIONS (FPT)-43692.3333333333-6:2Fl...,259452.0,ANODIZE SOLUTIONS (FPT),NaN,2019-08-15 08:00:00,PFAS-CL,6:2Fluorotelomersulfonic Acid (Report),NaN,<19.0,ppt_wt_v,...,8.0,Metal Finisher B,NaN,Metal Finishing,NaN,NaN,NaN,NaN,NaN,NaN
4,ANODIZE SOLUTIONS (FPT)-43692.3333333333-8:2Fl...,259452.0,ANODIZE SOLUTIONS (FPT),NaN,2019-08-15 08:00:00,PFAS-CL,8:2Fluorotelomersulfonic Acid (Report),NaN,<9.40,ppt_wt_v,...,10.0,Metal Finisher B,NaN,Metal Finishing,NaN,NaN,NaN,NaN,NaN,NaN


### Import Other Data

In [6]:
# Import PFAS Attributes

#Specify where the file is saved use '/' instead of '\'
excel_file_path = 'U:/Research_and_Innovation/_ModellingGroupProjects/PFAS Investigations/Data and analysis/Old Databases and Spreadsheets/Database as of 03-25-2024.xlsm'
#Specify sheet name
sheet_name = 'PFAS Attributes'

df_PFAS_Attributes = pd.read_excel(excel_file_path, sheet_name=sheet_name)

In [7]:
# Create a dictionary mapping PFAS Compounds to their corresponding Abbreviation (No need to change)
pfas_mapping = dict(zip(df_PFAS_Attributes['Database'], df_PFAS_Attributes['Abbrev']))

In [8]:
# Load the Excel file into a DataFrame
df = pd.read_excel("U:/Research_and_Innovation/_ModellingGroupProjects/PFAS Investigations/Data and analysis/PFAS Toolbox_ColorsPalette.xlsx",sheet_name='ColorPalette_NonTOP')

def parse_rgb(rgb_string):
    # Check if rgb_string is a string and not empty
    if isinstance(rgb_string, str) and rgb_string:
        # Remove parentheses and split the string by commas
        rgb_values = rgb_string.strip('()').split(',')
        # Convert each value to float and return as a tuple
        return tuple(int(value)/255 for value in rgb_values)
    else:
        # Handle the case where rgb_string is not a string or is empty
        return None  # or return (0, 0, 0, 0) for a default transparent color, for example



# Create a dictionary with PFAS compounds as keys and corresponding colors as values
compound_colors_dict = {compound: parse_rgb(rgb) for compound, rgb in zip(df['Database'], df['New_RGB'])}
compound_colors_dict_abbrev = {abbrev: parse_rgb(rgb) for abbrev, rgb in zip(df['Abbrev'], df['New_RGB'])}

### Data Records for Sampling Location

#### Functions

In [34]:
def create_custom_pivot_table(data_df, value_column='Quant Incl Estimates Incl Qual Flags', filter_qual_summary=None, start_date=None, end_date=None, filter_pfas_compounds=None, filter_sample_location=None):
    """
    Create a custom pivot table with filters based on 'Qual Summary (Grade for filtering)', 'Sample Location',
    'Sample Date', 'PFAS Compounds', and 'Sample Location'.

    Args:
        data_df (pd.DataFrame): The DataFrame containing the data.
        value_column (str, optional): The column to use as pivot table values (default is 'Quant Incl Estimates Incl Qual Flags').
        filter_qual_summary (list of str, optional): List of 'Qual Summary (Grade for filtering)' values to filter the data (default is None).
        start_date (str, optional): Start date for the date range filter (default is None).
        end_date (str, optional): End date for the date range filter (default is None).
        filter_pfas_compounds (list of str, optional): List of 'PFAS Compounds' values to filter the data (default is None).
        filter_sample_location (list of str, optional): List of 'Sample Location' values to filter the data (default is None).

    Returns:
        pd.DataFrame: The custom pivot table with average values.
    """
    try:
        # Apply filters based on 'Qual Summary (Grade for filtering)'
        filtered_data = data_df
        if filter_qual_summary:
            filtered_data = filtered_data[filtered_data['Qual Summary (Grade for filtering)'].isin(filter_qual_summary)]

        # Apply date range filter based on 'Sample Date'
        if start_date and end_date:
            filtered_data = filtered_data[(filtered_data['Sample Date'] >= start_date) & (filtered_data['Sample Date'] <= end_date)]

        # Apply filters based on 'PFAS Compounds'
        if filter_pfas_compounds:
            filtered_data = filtered_data[filtered_data['PFAS Compound'].isin(filter_pfas_compounds)]

        # Apply filter based on 'Sample Location'
        if filter_sample_location:
            filtered_data = filtered_data[filtered_data['Sample Location'].isin(filter_sample_location)]

        # Create a pivot table with 'Sample Date' as columns, 'PFAS Compound' as rows, and specified value column as values
        pivot_table = pd.pivot_table(filtered_data, values=value_column,
                                     index='PFAS Compound', columns='Sample Date',
                                     aggfunc='mean', fill_value=np.nan)

        return pivot_table

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

In [35]:
def create_combined_pivot(Quat, QuatInc, MDL):
    # Create an empty DataFrame with the same structure as the input tables
    combined_pivot = pd.DataFrame(index=Quat.index, columns=Quat.columns)
    
    # Iterate through rows and columns
    for row in Quat.index:
        for col in Quat.columns:
            Quat_value = Quat.loc[row, col] if row in Quat.index and col in Quat.columns else np.nan
            QuatInc_value = QuatInc.loc[row, col] if row in QuatInc.index and col in QuatInc.columns else np.nan
            MDL_value = MDL.loc[row, col] if row in MDL.index and col in MDL.columns else np.nan
            
            # Check if the values are numeric
            if pd.notna(Quat_value):
                Quat_value = float(Quat_value)  # Convert to float
                QuatInc_value = float(QuatInc_value)  # Convert to float
                MDL_value = float(MDL_value)  # Convert to float
                
                if Quat_value > 0:
                    combined_pivot.loc[row, col] = str(Quat_value)
                elif QuatInc_value > 0:
                    combined_pivot.loc[row, col] = str(QuatInc_value) + " E"
                else:
                    combined_pivot.loc[row, col] = "< " + str(MDL_value)
            else:
                combined_pivot.loc[row, col] = ""
    
    return combined_pivot

### Jireh PFAS Report

#### Filters and Settings

In [41]:
# Filter criteria
#filter_qual_summary (list of str, optional): List of 'Qual Summary (Grade for filtering)' values to filter the data (default is None).
filter_qual_summary = ['Good','Probably Ok','']
# filter_sample_location (list of str, optional): List of 'Sample Location' values to filter the data (default is None).
filter_sample_location=['Jireh Semiconductor Inc']
start_date='2022-04-12'
end_date='2024-12-30'

#### Pivot Table for Average of Quant Only Incl Qual Flags

In [42]:
quant_pivot=create_custom_pivot_table(data_df,value_column='Quant Only Incl Qual Flags', filter_qual_summary=filter_qual_summary, start_date=start_date, end_date=end_date, filter_pfas_compounds=None, filter_sample_location=filter_sample_location)
quant_pivot

Sample Date,2022-04-13,2022-08-04,2023-01-26,2023-04-07,2023-09-11,2023-12-06,2024-03-12
PFAS Compound,,,,,,,
10:2Fluorotelomersulfonic acid,0.0,0.0,0.0,0.0,0.0,0.00,NaN
11Cl-PF3OUdS,0.0,0.0,0.0,0.0,0.0,0.00,0.0
3:3 FTCA,NaN,NaN,NaN,NaN,NaN,0.00,0.0
4:2 Fluorotelomersulfonic acid,0.0,0.0,0.0,0.0,0.0,0.00,0.0
5:3 FTCA,NaN,NaN,NaN,NaN,NaN,0.00,0.0
6:2 Fluorotelomersulfonic acid,0.0,NaN,0.0,0.0,0.0,0.00,0.0
7:3 FTCA,NaN,NaN,NaN,NaN,NaN,0.00,0.0
8:2 Fluorotelomersulfonic acid,0.0,0.0,0.0,0.0,0.0,0.00,0.0
9Cl-PF3ONS,0.0,0.0,0.0,0.0,0.0,0.00,0.0


#### Pivot Table for Quant Incl Estimates Incl Qual Flags

In [43]:
QuatInc_Pivot = create_custom_pivot_table(data_df,value_column='Quant Incl Estimates Incl Qual Flags', filter_qual_summary=filter_qual_summary, start_date=start_date, end_date=end_date, filter_pfas_compounds=None, filter_sample_location=filter_sample_location)
QuatInc_Pivot

Sample Date,2022-04-13,2022-08-04,2023-01-26,2023-04-07,2023-09-11,2023-12-06,2024-03-12
PFAS Compound,,,,,,,
10:2Fluorotelomersulfonic acid,0.0,0.0,0.00,0.0,0.0,0.00,NaN
11Cl-PF3OUdS,0.0,0.0,0.00,0.0,0.0,0.00,0.00
3:3 FTCA,NaN,NaN,NaN,NaN,NaN,0.00,0.00
4:2 Fluorotelomersulfonic acid,0.0,0.0,0.00,0.0,0.0,0.00,0.00
5:3 FTCA,NaN,NaN,NaN,NaN,NaN,0.00,0.00
6:2 Fluorotelomersulfonic acid,0.0,NaN,0.00,0.0,0.0,0.00,0.00
7:3 FTCA,NaN,NaN,NaN,NaN,NaN,0.00,0.00
8:2 Fluorotelomersulfonic acid,0.0,0.0,0.00,0.0,0.0,0.00,0.00
9Cl-PF3ONS,0.0,0.0,0.00,0.0,0.0,0.00,0.00


#### Pivot Table for Avg MDL

In [44]:
MDL_Pivot=create_custom_pivot_table(data_df,value_column='MDL', filter_qual_summary=filter_qual_summary, start_date=start_date, end_date=end_date, filter_pfas_compounds=None, filter_sample_location=filter_sample_location)
MDL_Pivot

Sample Date,2022-04-13,2022-08-04,2023-01-26,2023-04-07,2023-09-11,2023-12-06,2024-03-12
PFAS Compound,,,,,,,
10:2Fluorotelomersulfonic acid,0.87,10.0,0.77,2.00,0.74,0.750,NaN
11Cl-PF3OUdS,0.44,5.0,0.48,1.30,0.47,1.185,1.80
3:3 FTCA,NaN,NaN,NaN,NaN,NaN,1.400,1.40
4:2 Fluorotelomersulfonic acid,0.44,5.0,0.48,1.30,0.47,1.035,1.50
5:3 FTCA,NaN,NaN,NaN,NaN,NaN,9.400,9.10
6:2 Fluorotelomersulfonic acid,1.70,NaN,0.48,1.30,0.47,1.435,2.30
7:3 FTCA,NaN,NaN,NaN,NaN,NaN,9.400,9.10
8:2 Fluorotelomersulfonic acid,0.87,10.0,0.58,1.50,0.56,1.480,2.40
9Cl-PF3ONS,0.44,5.0,0.48,1.30,0.47,0.705,0.91


#### Data Copillation Table

In [45]:
# Call function and provide previous tables as inputs
PFAS_Results_Location = create_combined_pivot(Quat=quant_pivot, QuatInc=QuatInc_Pivot, MDL=MDL_Pivot)

# Create a dictionary mapping PFAS Compounds to their corresponding Abbreviation (No need to change)
pfas_mapping = dict(zip(df_PFAS_Attributes['Database'], df_PFAS_Attributes['Abbrev']))

# Rename the index of data_df using the mapping (No need to change)
PFAS_Results_Location.index = PFAS_Results_Location.index.map(pfas_mapping)

PFAS_Results_Location

Sample Date,2022-04-13,2022-08-04,2023-01-26,2023-04-07,2023-09-11,2023-12-06,2024-03-12
PFAS Compound,,,,,,,
10:2FTS,< 0.87,< 10.0,< 0.77,< 2.0,< 0.74,< 0.75,
11Cl-PF3OUdS,< 0.44,< 5.0,< 0.48,< 1.3,< 0.47,< 1.185,< 1.8
3:3FTCA,,,,,,< 1.4,< 1.4
4:2FTS,< 0.44,< 5.0,< 0.48,< 1.3,< 0.47,< 1.0350000000000001,< 1.5
5:3FTCA,,,,,,< 9.4,< 9.1
6:2FTS,< 1.7,,< 0.48,< 1.3,< 0.47,< 1.435,< 2.3
7:3FTCA,,,,,,< 9.4,< 9.1
8:2FTS,< 0.87,< 10.0,< 0.58,< 1.5,< 0.56,< 1.48,< 2.4
9Cl-PF3ONS,< 0.44,< 5.0,< 0.48,< 1.3,< 0.47,< 0.705,< 0.91


#### Export Data Records

In [40]:
#dfs: A dictionary where keys are the tab names and values are the DataFrames to write.
dfs = {'Quant': quant_pivot,
       'QuantIncEstimate' :QuatInc_Pivot,
       'MDLs' : MDL_Pivot,
       'PFAS Report' : PFAS_Results_Location}

#filename: The name of the Excel file to write to.
filename= 'Jireh_PFAS Report.xlsx'
dfs_to_excel(dfs, filename)

KeyboardInterrupt: 

### JAE PFAS Report

#### Filters and Settings

In [27]:
# Filter criteria
#filter_qual_summary (list of str, optional): List of 'Qual Summary (Grade for filtering)' values to filter the data (default is None).
filter_qual_summary = ['Good','Probably Ok','']
# filter_sample_location (list of str, optional): List of 'Sample Location' values to filter the data (default is None).
filter_sample_location=['JAE Oregon Inc']


#### Pivot Table for Average of Quant Only Incl Qual Flags

In [28]:
quant_pivot=create_custom_pivot_table(data_df,value_column='Quant Only Incl Qual Flags', filter_qual_summary=filter_qual_summary, start_date=None, end_date=None, filter_pfas_compounds=None, filter_sample_location=filter_sample_location)
quant_pivot

Sample Date,2022-04-12,2022-08-04,2023-01-26,2023-04-06,2023-09-12,2023-12-06,2024-03-13
PFAS Compound,,,,,,,
10:2Fluorotelomersulfonic acid,NaN,0.0,0.0,0.0,0.0,NaN,NaN
11Cl-PF3OUdS,NaN,0.0,0.0,0.0,0.0,0.0,0.0
3:3 FTCA,NaN,NaN,NaN,NaN,NaN,0.0,0.0
4:2 Fluorotelomersulfonic acid,NaN,0.0,0.0,0.0,0.0,0.0,0.0
5:3 FTCA,NaN,NaN,NaN,NaN,NaN,0.0,0.0
6:2 Fluorotelomersulfonic acid,NaN,NaN,0.0,0.0,0.0,0.0,0.0
7:3 FTCA,NaN,NaN,NaN,NaN,NaN,0.0,0.0
8:2 Fluorotelomersulfonic acid,NaN,0.0,0.0,0.0,0.0,0.0,0.0
9Cl-PF3ONS,NaN,0.0,0.0,0.0,0.0,0.0,0.0


#### Pivot Table for Quant Incl Estimates Incl Qual Flags

In [30]:
QuatInc_Pivot = create_custom_pivot_table(data_df,value_column='Quant Incl Estimates Incl Qual Flags', filter_qual_summary=filter_qual_summary, start_date=None, end_date=None, filter_pfas_compounds=None, filter_sample_location=filter_sample_location)
QuatInc_Pivot

Sample Date,2022-04-12,2022-08-04,2023-01-26,2023-04-06,2023-09-12,2023-12-06,2024-03-13
PFAS Compound,,,,,,,
10:2Fluorotelomersulfonic acid,NaN,0.0,0.0,0.00,0.00,NaN,NaN
11Cl-PF3OUdS,NaN,0.0,0.0,0.00,0.00,0.00,0.0
3:3 FTCA,NaN,NaN,NaN,NaN,NaN,0.00,0.0
4:2 Fluorotelomersulfonic acid,NaN,0.0,0.0,0.00,0.00,0.00,0.0
5:3 FTCA,NaN,NaN,NaN,NaN,NaN,0.00,0.0
6:2 Fluorotelomersulfonic acid,NaN,NaN,0.0,0.80,2.40,0.00,0.0
7:3 FTCA,NaN,NaN,NaN,NaN,NaN,0.00,0.0
8:2 Fluorotelomersulfonic acid,NaN,0.0,0.0,0.00,0.00,0.00,0.0
9Cl-PF3ONS,NaN,0.0,0.0,0.00,0.00,0.00,0.0


#### Pivot Table for MDL

In [31]:
MDL_Pivot=create_custom_pivot_table(data_df,value_column='MDL', filter_qual_summary=filter_qual_summary, start_date=None, end_date=None, filter_pfas_compounds=None, filter_sample_location=filter_sample_location)
MDL_Pivot

Sample Date,2022-08-04,2023-01-26,2023-04-06,2023-09-12,2023-12-06,2024-03-13
PFAS Compound,,,,,,
10:2Fluorotelomersulfonic acid,10.0,8.0,0.73,0.68,NaN,NaN
11Cl-PF3OUdS,5.0,5.0,0.46,0.42,21.00,1.90
3:3 FTCA,NaN,NaN,NaN,NaN,16.00,1.40
4:2 Fluorotelomersulfonic acid,5.0,5.0,0.46,0.42,18.00,16.00
5:3 FTCA,NaN,NaN,NaN,NaN,110.00,9.50
6:2 Fluorotelomersulfonic acid,NaN,5.0,NaN,NaN,27.00,24.00
7:3 FTCA,NaN,NaN,NaN,NaN,110.00,9.50
8:2 Fluorotelomersulfonic acid,10.0,6.0,0.55,0.51,2.80,2.50
9Cl-PF3ONS,5.0,5.0,0.46,0.42,11.00,0.95


#### Data Copillation

In [32]:
# Call function and provide previous tables as inputs
PFAS_Results_Location = create_combined_pivot(Quat=quant_pivot, QuatInc=QuatInc_Pivot, MDL=MDL_Pivot)

# Create a dictionary mapping PFAS Compounds to their corresponding Abbreviation (No need to change)
pfas_mapping = dict(zip(df_PFAS_Attributes['Database'], df_PFAS_Attributes['Abbrev']))

# Rename the index of data_df using the mapping (No need to change)
PFAS_Results_Location.index = PFAS_Results_Location.index.map(pfas_mapping)

PFAS_Results_Location

Sample Date,2022-04-12,2022-08-04,2023-01-26,2023-04-06,2023-09-12,2023-12-06,2024-03-13
PFAS Compound,,,,,,,
10:2FTS,,< 10.0,< 8.0,< 0.73,< 0.68,,
11Cl-PF3OUdS,,< 5.0,< 5.0,< 0.46,< 0.42,< 21.0,< 1.9
3:3FTCA,,,,,,< 16.0,< 1.4
4:2FTS,,< 5.0,< 5.0,< 0.46,< 0.42,< 18.0,< 16.0
5:3FTCA,,,,,,< 110.0,< 9.5
6:2FTS,,,< 5.0,0.8 E,2.4 E,< 27.0,< 24.0
7:3FTCA,,,,,,< 110.0,< 9.5
8:2FTS,,< 10.0,< 6.0,< 0.55,< 0.51,< 2.8,< 2.5
9Cl-PF3ONS,,< 5.0,< 5.0,< 0.46,< 0.42,< 11.0,< 0.95


#### Export Data

In [33]:
#dfs: A dictionary where keys are the tab names and values are the DataFrames to write.
dfs = {'Quant': quant_pivot,
       'QuantIncEstimate' :QuatInc_Pivot,
       'MDLs' : MDL_Pivot,
       'PFAS Report' : PFAS_Results_Location}

#filename: The name of the Excel file to write to.
filename= 'JAE_PFAS Report.xlsx'
dfs_to_excel(dfs, filename)

File 'JAE_PFAS Report.xlsx' has been written with 4 tabs.
